In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import seaborn as sns
import copy
import scipy.stats as stats

D:\anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
D:\anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def set_feature_dict(column_names,feature_dict): # Assigning quantitative information
    for feature in column_names:
        feature_dict[feature] = {'change_range':0,'correlation': False,'target':0}
    return feature_dict

In [3]:
def Feature_correlation_analysis(column_names,feature_dict,spearmanr_data): # Correlation between features
    Feature_correlation_list = []
    correlation_matrix, _ = spearmanr(spearmanr_data, axis=0)
    threshold = 0.7
    correlation_matrix[np.abs(correlation_matrix) < threshold] = np.nan
    plt.figure(figsize=(10, 8)) 
    mask = np.tri(*correlation_matrix.shape, k=-1)
    correlation_matrix = np.ma.array(correlation_matrix, mask=mask)
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", square=True)
    plt.xticks(range(len(column_names)), column_names, rotation=45, horizontalalignment='right')
    plt.yticks(range(len(column_names)), column_names)
    plt.title("Spearman")
    rela_class = []
    count = -1
    rows, cols = np.where(~np.isnan(correlation_matrix))
    for row, col in zip(rows, cols):
        if row != col and correlation_matrix[row, col] != 'masked':
            Feature_correlation_list.append(str(f"Feature {column_names[row]} and Feature {column_names[col]} have correlation: {correlation_matrix[row, col]:.2f}"))
            feature1 = column_names[row]
            feature2 = column_names[col]
            if not rela_class:
                rela_class.append([feature1,feature2])
                continue
            for i in rela_class:
                flag = 0
                count += 1
                if feature1 in i and feature2 not in i :
                    rela_class[count].append(feature2)
                    flag = 1
                    break
                elif feature2 in i and feature1 not in i:
                    rela_class[count].append(feature1)
                    flag = 1
                    break
                elif feature1 in i and feature2 in i:
                    flag = 1
                    break
            if flag == 0:
                rela_class.append([feature1, feature2])
            flag = 0
            count = -1
    result = []
    for sublist in rela_class:
        merged = False
        for existing in result:
            if any(item in existing for item in sublist):
                existing.extend(item for item in sublist if item not in existing)
                merged = True
                break
        if not merged:
            result.append(sublist)
    rela_class = result
    for i in rela_class:
        for j in i:
            if j in feature_dict:
                feature_dict[j]['correlation'] = True
    F = column_names
    F_copy = copy.deepcopy(F)
    for i in F:
        if feature_dict[i]['correlation'] is False:
            feature_dict[i]['NOFR'] = 0
        else:
            for j in rela_class:
                for metric in j:
                    if metric == i:
                         feature_dict[i]['NOFR'] = len(j)-1    
    return rela_class,feature_dict,Feature_correlation_list

In [4]:
def target_correlation_analysis(feature_dict,target_variable,column_names): # Correlation between features and labels
    target_variable_list = []
    unique_values, counts = np.unique(target_variable.values, return_counts=True)
    if unique_values.shape !=(2,):
        for feature in column_names:  
            point_biserial_corr, p_value = spearmanr(df[feature], target_variable)
            feature_dict[feature]['target'] = point_biserial_corr
            target_variable_list.append(str(f"{feature} - target_variable: {point_biserial_corr:.2f}"))
    else:
        for feature in column_names: 
            point_biserial_corr, p_value = stats.pointbiserialr(df[feature], target_variable)
            feature_dict[feature]['target'] = point_biserial_corr
            target_variable_list.append(str(f"{feature} - target_variable: {point_biserial_corr:.2f}"))
    return feature_dict,target_variable_list